# Model Training Example


In [2]:
# Load libraries for creating a xgboost model of the iris dataset 
from sklearn import datasets
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

In [3]:
# Load the iris data as an example
# NOTE: For the assignment, you will need to use the data from Kaggle 
#       and process the text data into a usable format and then create 
#       a ML model that satisfies the assignment task!
iris = datasets.load_iris() 
X = iris.data                
y = iris.target             

In [4]:
# Split data into test and train 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [5]:
# Create XGBoost classifier 
xgbClf = XGBClassifier(use_label_encoder = False)
xgbClf.fit(X_train, y_train)
xgbClf.save_model("model.json")

[18:54:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


# Model Deployment via Jupyter Notebook

In [9]:
# Load workspace
from azureml.core import Workspace
ws = Workspace.from_config(path = "config.json")
print(ws)

Performing interactive authentication. Please follow the instructions on the terminal.
Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace.create(name='MSA-ModelDeployment', subscription_id='1728fb6f-0674-4adc-b1a8-0462995e20a5', resource_group='MSA-ModelDeployment')


In [10]:
from azureml.core.model import Model

# Register model
model = Model.register(ws, model_name = "iris-xgboost", model_path = "model.json")

Registering model iris-xgboost


In [12]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig

# Create environment
# env = Environment.from_existing_conda_environment(name = "iris-xgboost",
                                                # conda_environment_name = "azure")

env = Environment(name = "iris-xgboost")
conda_dep = CondaDependencies()
conda_dep.add_conda_package("numpy")
conda_dep.add_conda_package("xgboost")
env.python.conda_dependencies = conda_dep

dummy_inference_config = InferenceConfig(
    environment = env,
    source_directory = "./source_dir",
    entry_script = "./echo_score.py",
)

In [13]:
from azureml.core.webservice import AciWebservice
aci_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
service = Model.deploy(
    ws,
    "iris-xgboost",
    [model],
    dummy_inference_config,
    aci_config,
    overwrite = True,
)
service.wait_for_deployment(show_output = True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-07-26 18:59:24+12:00 Creating Container Registry if not exists..
2021-07-26 18:59:39+12:00 Registering the environment..
2021-07-26 18:59:42+12:00 Building image..
2021-07-26 19:10:28+12:00 Generating deployment configuration.
2021-07-26 19:10:29+12:00 Submitting deployment to compute.
2021-07-26 19:10:39+12:00 Checking the status of deployment iris-xgboost.
2021-07-26 19:13:37+12:00 Checking the status of inference endpoint iris-xgboost.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [16]:
import requests
import json

uri = service.scoring_uri
requests.get("<API Endpoint Here>")
headers = {"Content-Type": "application/json"}
data = {
    "data": [[6.1, 2.8, 4.7, 1.2]],
}
data = json.dumps(data)
response = requests.post(uri, data = data, headers = headers)
print(response.json())

['versicolor']
